## Pixel

In [1]:
import math


class Pixel:
    """
    Dont forget to set class variables 'is_a_star' and 'target' before using this class.
    """

    is_a_star = False  # False -> Best First Search, True-> A*
    target = (5, 7)  # Target to which we calculate the Heuristic Function

    def __init__(self, x, y, color, parent_cost):
        self.x = x
        self.y = y
        self.color = color
        self.cost = parent_cost + Pixel.get_pixel_cost(color)

        
    # Evaluating Pixel Values

    def h(self):
        """ Heuristic Function"""
        return Pixel.manhattan_distance(self.x, self.y, Pixel.target[0], Pixel.target[1])

    def g(self):
        """ Cost Function"""
        return self.cost

    def f(self):
        if not Pixel.is_a_star:
            return self.h()  # -> Best First Search
        return self.h() + self.g()  # -> A *

    
    # Comparing Pixels

    def __lt__(self, other):
        return self.f() < other.f()

    def __le__(self, other):
        return self.f() <= other.f()

    def __gt__(self, other):
        return self.f() > other.f()

    def __ge__(self, other):
        return self.f() >= other.f()

    def __eq__(self, other):
        return self.f() == other.f()

    def __ne__(self, other):
        return self.f() != other.f()

    
    # Printing Pixels

    def __repr__(self):
        return f"(x={self.x},y={self.y},color={self.color})"

    def __str__(self):
        return f"(x={self.x},y={self.y},color={self.color})"
    
    
    # Utility Methods
    
    @staticmethod
    def get_pixel_cost(color):
        if color[0] == 0:
            return 1
        return 1 / (color[0])  # Reverse of the 'red' in RGB   

    @staticmethod
    def euclidean_distance(x1, y1, x2, y2):
        return math.sqrt((x1 ** 2 - x2 ** 2) + (y1 ** 2 - y2 ** 2))

    @staticmethod
    def manhattan_distance(x1, y1, x2, y2):
        return abs(x1 - x2) + abs(y1 - y2)


In [2]:
p = Pixel(3,5,color=(2,3,5), parent_cost=0)
k = Pixel(3,5,color=(2,3,5), parent_cost=50)

In [3]:
Pixel.is_a_star = False

In [4]:
p == k

True

In [5]:
Pixel.is_a_star = True

In [6]:
p == k

False

## Priority Queue of Pixels

In [7]:
class PriorityQueue:
    
    def __init__(self, initial_elements: list, comparator, max_redundant_element_count=100):
        self.comparator = comparator
        self.max_redundant_element_count = max_redundant_element_count    # See delete method
        self.heap = self.heapify(initial_elements)
        self.length = len(self.heap)

    def heapify(self, elements: list):
        """
        Takes list of elements, returns Heap
        Complexity: O(n)   
        """
        last_index = len(elements) - 1  # Get Last Element's index
        parent_of_last_index = (last_index - 1) // 2  # Get Last Element's Parent's index
        # After last parent the tree, there is no subtree to adjust 
        for i in range(parent_of_last_index, -1, -1):
            # Starting from last parent, assuming the given 'i' as the root of the subtree, down_heap
            self.down_heap(elements, len(elements), root_index=i)
        return elements  # Return the heap

    def insert(self, x: int):
        """
         Complexity: O(logN)
        """
        i = self.length  # index of the x
        parent_i = (i - 1) // 2  # index of x's parent      
        
        self.heap.append(x)  # Insert x into its index
        self.length += 1

        while i > 0 and self.comparator(x, self.heap[parent_i]):
            self.heap[i] = self.heap[parent_i]  # put the parent into x's place
            i = parent_i  # Update x's index, hypothetically x is there
            parent_i = (i - 1) // 2  # Calculate x's new parent's index
        self.heap[i] = x  # Insert x into its real place

    def delete(self):
        """
        Complexity = O(logN)
        """

        deleted_element = self.heap[0]  # Delete the root by taking and putting last el into its place
        self.heap[0] = self.heap[self.length - 1]  # Put the last element into root of the tree
        self.down_heap(self.heap, length=self.length)  # Adjust the tree so that it becomes heap again
        self.length -= 1  # Exclude last element since it is no longer inside the heap
        
        # If list grows so much, deallocate the unnecessary space
        actual_length = len(self.heap)
        if (actual_length - self.length) > self.max_redundant_element_count:
            self.heap = self.heap[:self.length]

        return deleted_element

    def down_heap(self, heap: list, length, root_index=0):
        i = root_index  # Start from given subtrees root
        max_child = self.get_max_priority_child_index(heap, length, i)  # Get max priority child's index
        while (max_child is not None) and self.comparator(heap[max_child], heap[i]):
            # while child exists and child greater than parent, swap them
            heap[i], heap[max_child] = heap[max_child], heap[i]  # Swap
            i = max_child  # Update node's index
            max_child = self.get_max_priority_child_index(heap, length, i)

    def get_max_priority_child_index(self, heap, length, index):
        last_index = length - 1
        if last_index < (index * 2 + 1):  # No child
            return None
        elif last_index < (index * 2 + 2):  # Only Left Child exists
            return index * 2 + 1
        else:  # Both children exists
            if self.comparator(heap[index * 2 + 1], heap[index * 2 + 2]):
                return index * 2 + 1  # Return Left Child as Max Priority child
            else:
                return index * 2 + 2  # Return Right Child as Max Priority child

    def __repr__(self):
        result = "[\n"
        for i in range(0,self.length):
            result += str(i) + ":" + str(self.heap[i]) + "\n"
        result += "]"
        return result

    def __str__(self):
        return str(self.heap[:self.length])



In [8]:
p1 = Pixel(3,5,color=(8,3,5), parent_cost=0)
p2 = Pixel(7,8,color=(6,3,5), parent_cost=1)
p3 = Pixel(8,8,color=(3,3,5), parent_cost=2)
p4 = Pixel(9,8,color=(10,3,5), parent_cost=3)
p5 = Pixel(10,8,color=(5,3,5), parent_cost=4)
p6 = Pixel(11,8,color=(4,3,5), parent_cost=5)
p7 = Pixel(12,8,color=(9,3,5), parent_cost=6)

In [9]:
queue = PriorityQueue([p1,p2,p3,p4,p5,p6,p7,], lambda x,y:x < y)  # Min Priority Queue

In [10]:
queue

[
0:(x=3,y=5,color=(8, 3, 5))
1:(x=7,y=8,color=(6, 3, 5))
2:(x=8,y=8,color=(3, 3, 5))
3:(x=9,y=8,color=(10, 3, 5))
4:(x=10,y=8,color=(5, 3, 5))
5:(x=11,y=8,color=(4, 3, 5))
6:(x=12,y=8,color=(9, 3, 5))
]

In [11]:
queue.insert(  Pixel(13,8,color=(7,3,5), parent_cost=14) )

In [12]:
queue

[
0:(x=3,y=5,color=(8, 3, 5))
1:(x=7,y=8,color=(6, 3, 5))
2:(x=8,y=8,color=(3, 3, 5))
3:(x=9,y=8,color=(10, 3, 5))
4:(x=10,y=8,color=(5, 3, 5))
5:(x=11,y=8,color=(4, 3, 5))
6:(x=12,y=8,color=(9, 3, 5))
7:(x=13,y=8,color=(7, 3, 5))
]

In [13]:
queue.delete()

(x=3,y=5,color=(8, 3, 5))

In [14]:
queue

[
0:(x=7,y=8,color=(6, 3, 5))
1:(x=9,y=8,color=(10, 3, 5))
2:(x=8,y=8,color=(3, 3, 5))
3:(x=13,y=8,color=(7, 3, 5))
4:(x=10,y=8,color=(5, 3, 5))
5:(x=11,y=8,color=(4, 3, 5))
6:(x=12,y=8,color=(9, 3, 5))
]